In [ ]:
import requests
import uuid
from urllib.parse import urlencode

## create one from this url: https://developer.spotify.com/dashboard
client_id = '<your-client-id>'
client_secret = "<your-client-secret>"
redirect_uri = 'localhost:8888/callback'

state = uuid.uuid4().hex
scope = 'user-read-private user-read-email user-library-read'

url_data = {
  "grant_type": "client_credentials",
  "response_type": "token",
  "client_id": client_id,
  "client_secret": client_secret,
  "scope": scope,
  "state": state,
  "redirect_uri": redirect_uri
}


url = f"https://accounts.spotify.com/authorize?{urlencode(url_data)}"  
url
# click the output of this cell to perform authentication and get the access token

In [45]:
access_token = "<redacted>" # you can have the access token after authenticating from the prepared url in the previous step

data = []
url = "https://api.spotify.com/v1/me/tracks?market=ID&limit=50&offset=0"
while url is not None:
    print("Fetching data from url", url)
    resp = requests.get(
        url=url,
        headers={
            "Authorization": f"Bearer {access_token}"
        }
    )
    data.extend(resp.json()['items'])
    url = resp.json()['next']

data

Fetching data from url https://api.spotify.com/v1/me/tracks?market=ID&limit=50&offset=0
Fetching data from url https://api.spotify.com/v1/me/tracks?offset=50&limit=50&market=ID
Fetching data from url https://api.spotify.com/v1/me/tracks?offset=100&limit=50&market=ID
Fetching data from url https://api.spotify.com/v1/me/tracks?offset=150&limit=50&market=ID
Fetching data from url https://api.spotify.com/v1/me/tracks?offset=200&limit=50&market=ID
Fetching data from url https://api.spotify.com/v1/me/tracks?offset=250&limit=50&market=ID
Fetching data from url https://api.spotify.com/v1/me/tracks?offset=300&limit=50&market=ID
Fetching data from url https://api.spotify.com/v1/me/tracks?offset=350&limit=50&market=ID
Fetching data from url https://api.spotify.com/v1/me/tracks?offset=400&limit=50&market=ID
Fetching data from url https://api.spotify.com/v1/me/tracks?offset=450&limit=50&market=ID
Fetching data from url https://api.spotify.com/v1/me/tracks?offset=500&limit=50&market=ID
Fetching data

[{'added_at': '2024-01-16T23:14:00Z',
  'track': {'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3WwGRA2o4Ux1RRMYaYDh7N'},
      'href': 'https://api.spotify.com/v1/artists/3WwGRA2o4Ux1RRMYaYDh7N',
      'id': '3WwGRA2o4Ux1RRMYaYDh7N',
      'name': 'Mariya Takeuchi',
      'type': 'artist',
      'uri': 'spotify:artist:3WwGRA2o4Ux1RRMYaYDh7N'}],
    'external_urls': {'spotify': 'https://open.spotify.com/album/4m5Rko9RpsBpLS0PsX8zIN'},
    'href': 'https://api.spotify.com/v1/albums/4m5Rko9RpsBpLS0PsX8zIN',
    'id': '4m5Rko9RpsBpLS0PsX8zIN',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/ab67616d0000b273c62c781adc393e54d8674e45',
      'width': 640},
     {'height': 300,
      'url': 'https://i.scdn.co/image/ab67616d00001e02c62c781adc393e54d8674e45',
      'width': 300},
     {'height': 64,
      'url': 'https://i.scdn.co/image/ab67616d00004851c62c781adc393e54d8674e45',
      'width': 64}],
    'is_p

In [49]:
columns = list(set(
    col for record in data for col in record['track'].keys()
))
columns

['id',
 'duration_ms',
 'name',
 'type',
 'track_number',
 'explicit',
 'is_playable',
 'disc_number',
 'external_urls',
 'external_ids',
 'restrictions',
 'artists',
 'is_local',
 'popularity',
 'href',
 'linked_from',
 'uri',
 'album',
 'preview_url']

In [51]:
import pandas as pd

df = pd.DataFrame([
    {
        'track_id': record['track']['id'],
        'artists': ';'.join(map(lambda x: x['name'], record['track']['artists'])),
        'album_name': record['track']['album']['name'],
        'track_name': record['track']['name']
    }
    for record in data
])
df

,track_id,artists,album_name,track_name
0,7cVMaM7bLDXCXzkTMR0Yco,Mariya Takeuchi,LOVE SONGS,SEPTEMBER
1,0o9zmvc5f3EFApU52PPIyW,Fujii Kaze,HELP EVER HURT NEVER,Shinunoga E-Wa
2,3RvdkNMcSy71m0aT6UF9Uf,back number,ユーモア,水平線
3,0hh8HEObfyaRV7jZyVOPR1,Hiroji Miyamoto,SEPTEMBER,SEPTEMBER
4,1QNbkBKOiBILvRAzkyvtqC,Tokimeki Records;Hikari,悲しみがとまらない I CAN'T STOP THE LONLINESS (feat. ひかり),悲しみがとまらない I CAN'T STOP THE LONLINESS (feat. ひかり)
...,...,...,...,...
1044,2Z8WuEywRWYTKe1NybPQEW,Twenty One Pilots,Blurryface,Ride
1045,6JV2JOEocMgcZxYSZelKcc,Justin Timberlake,CAN'T STOP THE FEELING! (from DreamWorks Anima...,CAN'T STOP THE FEELING! (from DreamWorks Anima...
1046,6s3GEN8wK0OMzzzZbXj0fu,Coldplay;Seeb,Hymn for the Weekend (Seeb Remix),Hymn for the Weekend - Seeb Remix
1047,18tLvggTFEZuzLdcnuej2u,Rizky Febian,Kesempurnaan Cinta,Kesempurnaan Cinta


In [53]:
df.to_csv("./personal_saved_song.csv")

## Labeling the source dataset

The goal of the task is to create a recommendation who person (myself) is liking the song in Spotify. This will make the source dataset labeled based on the availability of the song in my personal liked songs. So, if the song is in my Spotify's liked song, it will be labeled as 1. Of course, not all song in my liked song are available in the source dataset due to the fact the dataset is last updated in years ago.

In [57]:
dataset = pd.read_csv('source_dataset.csv')
dataset

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113995,113995,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,...,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music
113996,113996,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,...,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music
113997,113997,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,...,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music
113998,113998,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,...,-10.889,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music


perform a labelling

In [59]:
dataset['is_liked'] = dataset['track_id'].map(lambda x: 1 if x in df['track_id'].to_list() else 0)
dataset

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,is_liked
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic,1
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic,0
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic,0
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic,0
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113995,113995,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,...,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music,0
113996,113996,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,...,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music,0
113997,113997,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,...,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music,0
113998,113998,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,...,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music,0


In [61]:
dataset.groupby('is_liked').count()['track_id']

is_liked
0    113458
1       542
Name: track_id, dtype: int64

In [62]:
dataset[dataset['is_liked'] == 1]

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,is_liked
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,...,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic,1
8,8,0IktbUcnAGrvD03AWnz3Q8,Jason Mraz;Colbie Caillat,We Sing. We Dance. We Steal Things.,Lucky,74,189613,False,0.625,0.414,...,1,0.0369,0.2940,0.000000,0.1510,0.669,130.088,4,acoustic,1
11,11,5ivF4eQBqJiVL5IAE9jRyl,Jason Mraz,Love Is a Four Letter Word,I Won't Give Up,69,240165,False,0.483,0.303,...,1,0.0429,0.6940,0.000000,0.1150,0.139,133.406,3,acoustic,1
71,71,3TwtrR1yNLY1PMPsrGQpOp,Five For Fighting,America Town,Superman (It's Not Easy),70,221693,False,0.382,0.416,...,1,0.0302,0.0733,0.000000,0.0719,0.125,102.089,4,acoustic,1
108,108,0ltIx8PLjtWoS9zn9PqsZW,Kitri,Hikare Inochi,Hikare Inochi,51,322146,False,0.592,0.769,...,1,0.0311,0.3190,0.000005,0.1370,0.385,102.033,4,acoustic,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107361,107361,5UHuuQGEgMbZrYnHjQG9ny,a-ha;Kygo,Take on Me (Kygo Remix),Take on Me - Kygo Remix,55,226666,False,0.535,0.780,...,1,0.1130,0.1590,0.000761,0.1370,0.622,179.964,4,synth-pop,1
109003,109003,6uBhi9gBXWjanegOb2Phh0,Zedd;Alessia Cara,Stay,Stay,76,210090,False,0.690,0.622,...,0,0.0622,0.2530,0.000000,0.1160,0.544,102.040,4,techno,1
109007,109007,09IStsImFySgyp0pIQdqAc,Zedd;Maren Morris;Grey,The Middle,The Middle,80,184732,False,0.753,0.657,...,1,0.0449,0.1710,0.000000,0.1120,0.437,107.010,4,techno,1
109009,109009,2syQ0f4SAQdzvlAewPl1DJ,Zedd;Liam Payne,Get Low (with Liam Payne),Get Low (with Liam Payne),57,204583,False,0.718,0.926,...,1,0.0494,0.0270,0.000018,0.0513,0.540,108.018,4,techno,1


In [63]:
dataset.to_csv('labeled_dataset.csv') 